# Time Stamp Extract

Brief 1-2 sentence description of notebook.

TODO: Supplement the description
- Notebook that extracts the timestamps and gets the time that tones played

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/projects/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
import utilities.helper
import trodes.read_exported

## Inputs & Data

- Explanation of each input and where it comes from.

Inputs and Required data loading
- input variable names are in all caps snake case
- Whenever an input changes or is used for processing 
- The variables are all lower in snake case

In [8]:
# Path of the directory that contains the Spike Gadgets recording and the exported timestamp files
# Exported with this tool https://docs.spikegadgets.com/en/latest/basic/ExportFunctions.html
# Export these files:
    # -raw – Continuous raw band export.
    # -dio – Digital IO channel state change export.
    # -analogio – Continuous analog IO export.
INPUT_DIR = "/scratch/back_up/reward_competition_extention/data"
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs
TONE_DIN = "dio_ECU_Din1"
TONE_STATE = 1
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [9]:
COLS_TO_KEEP = ['session_dir', 'recording', 'metadata_dir', 'metadata_file',
'original_file', 'filename', 'session_path', 'all_subjects',
       'current_subject', 'event_timestamps', 'video_name',
       'video_timestamps', 'event_frames', 'first_item_data']

In [10]:
RAW_COLS_TO_KEEP = ['session_dir',
 'recording',
 'original_file',
 'session_path',
 'current_subject',
 'first_item_data',
 'first_timestamp',
 'all_subjects']

In [11]:
STATE_COLS_TO_KEEP = ['session_dir',
 'metadata_file',
 'event_timestamps',
 'video_name',
 'video_timestamps',
 'event_frames',]

In [12]:
same_columns = ['session_dir', 'video_name']
different_columns = ['metadata_file', 'event_frames', 'event_timestamps']

In [13]:
# TODO: Find way not to hard code this
# ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/standard/2023_06_*/*.rec")
ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/omission/2023_06_*/*.rec")



In [14]:
ALL_SESSION_DIR

['/scratch/back_up/reward_competition_extention/data/omission/2023_06_17/20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_21/20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_20/20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_19/20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4.rec']

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Other documentation

raw directory
- raw_group0.dat
    - voltage_value: Array with voltage measurement for each channel at each timestamp
- timestamps.dat
    - voltage_time_stamp: The time stamp of each voltage measurement

parent directory
- 1.videoTimeStamps.cameraHWSync
    - frame_number: Calculated by getting the index of each video time stamp tuple 
    - PosTimestamp: The time stamp of each video frame
    - HWframeCount: Unknown value. Starts at 30742 and increases by 1 for each tuple  
    - HWTimestamp: Unknown value. All zeroes
    - video_time: Calculated by dividing the frame number by the fps(frames per second) 
    - video_seconds: video_time, but rounded to seconds  	
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_PosTimestamp 	
        - filledHWframeCount 	
        - filled_frame_number 	
        - filled_video_time 	
        - filled_video_seconds 	

DIO directory
- dio_ECU_Din1.dat
    - time: The time stamp the corresponds to the DIN input
    - state: Binary state of whether there is input from DIN or not 	
    - trial_number: Calculated by adding 1 to every time there is a DIN input
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_state 	
        - filled_trial_number

ss_output directory (Spike sorting with Spike interface)
- firings.npz
    - unit_id: All the units that had a spike train for the given timestamp 	
    - number_of_units: Calculated by counting the number of units that had a spike train

## Functions

- function names are short and in snake case all lowercase
- a function name should be unique but does not have to describe the function
- doc strings describe functions not function names

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [15]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# LOOP 1: Extracting all the Trodes

- Getting all the data from all the exported Trodes files

- Getting all the data from all the exported Trodes files and saving it to `session_to_trodes_data`
    - Creates a dictionary with the structure of:
        - `{dir_name: {file_name: metadata, file_name_2: metadata_2}, dir_name_2: {file_name_3: metadata_3, file_name_4: metadata_4}}`

In [16]:
# Saving the trodes data for each session
# Each key is a session name
# Each value is a dictionary of every recording file in that session
session_to_trodes_data = utilities.helper.create_recursive_dict()


# Saving the path of the session recording
session_to_path = {}

# Going through each session recording
# Which includes all the recordings from all the miniloggers and cameras
for session_path in ALL_SESSION_DIR:   
    try:
        # Getting the name of the session from the path
        session_basename = os.path.splitext(os.path.basename(session_path))[0]
        print("Current Session: {}".format(session_basename))
        # Reading the trodes data for every recording file in the session directory
        session_to_trodes_data[session_basename] = trodes.read_exported.organize_all_trodes_export(session_path)
        
        session_to_path[session_basename] = session_path
    except Exception as e: 
        print(e)


Current Session: 20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2


/nancy/projects/reward_competition_extention/src/trodes/read_exported.py:62: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(dtype_spec)


Skipping file 20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged.timestampoffset.txt due to error: Settings format not supported
Skipping file 20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged.timestampoffset.txt due to error: Settings format not supported
Current Session: 20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2
Skipping file 20230621_111240_standard_comp_to_omission_D5_subj_1-4_t3b3L_box1_merged.timestampoffset.txt due to error: Settings format not supported
Current Session: 20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1
Skipping file 20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged.timestampoffset.txt due to error: Settings format not supported
Skipping file 20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged.timestampoffset.txt due to error: Settings format not supported
Current Session: 20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1
Skipping fi

In [17]:
session_to_trodes_data

defaultdict(<function utilities.helper.create_recursive_dict()>,
            {'20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2': defaultdict(dict,
                         {'20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged': {'DIO': {'dio_ECU_Dout1': {'description': 'State change data for one digital channel. Display_order is 1-based',
                             'byte_order': 'little endian',
                             'original_file': '20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged.rec',
                             'clockrate': '20000',
                             'trodes_version': '2.4.0',
                             'compile_date': 'May 24 2023',
                             'compile_time': '10:11:04',
                             'qt_version': '6.2.2',
                             'commit_tag': 'heads/Release_2.4.0-0-g1eecf3b7',
                             'controller_firmware': '3.17',
                          

- Adding the video timestamps

In [18]:
for session_path in ALL_SESSION_DIR:   
    try:
        session_basename = os.path.splitext(os.path.basename(session_path))[0]
        print("Current Session: {}".format(session_basename))
        file_to_video_timestamps = {}
        for video_timestamps in glob.glob(os.path.join(session_path, "*cameraHWSync")):
            video_basename = os.path.basename(video_timestamps)
            print("Current Video Name: {}".format(video_basename))
            timestamp_array = trodes.read_exported.read_trodes_extracted_data_file(video_timestamps)
            if "video_timestamps" not in session_to_trodes_data[session_basename][session_basename]:
                session_to_trodes_data[session_basename][session_basename]["video_timestamps"] = defaultdict(dict)
            session_to_trodes_data[session_basename][session_basename]["video_timestamps"][video_basename.split(".")[-3]] = timestamp_array
    
    
    except Exception as e: 
        print(e)

Current Session: 20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2
Current Video Name: 20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.3.videoTimeStamps.cameraHWSync
Current Video Name: 20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.2.videoTimeStamps.cameraHWSync
Current Video Name: 20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.1.videoTimeStamps.cameraHWSync
Current Session: 20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2
Current Video Name: 20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.2.videoTimeStamps.cameraHWSync
Current Video Name: 20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.1.videoTimeStamps.cameraHWSync
Current Session: 20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1
Current Video Name: 20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.2.videoTimeStamps.cameraHWSync
Current Video Name: 20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.1.v

In [19]:
session_to_trodes_data[session_basename][session_basename]["video_timestamps"]

defaultdict(dict,
            {'4': {'clock rate': '30000',
              'fields': '<PosTimestamp uint32><HWframeCount uint32><HWTimestamp uint64>',
              'data': array([( 1642254, 0, 0), ( 1643640, 0, 0), ( 1645026, 0, 0), ...,
                     (67770406, 0, 0), (67770406, 0, 0), (67771792, 0, 0)],
                    dtype=[('PosTimestamp', '<u4'), ('HWframeCount', '<u4'), ('HWTimestamp', '<u8')]),
              'filename': '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4.4.videoTimeStamps.cameraHWSync'}})

- Creating a dataframe the dictionary with a column for:
  - Session directory
  - Recording name
  - Metadata directory
  - Metadata file
  - And a column for each metadata

In [20]:
# Creating a dataframe from the nested dictionary
trodes_metadata_df = pd.DataFrame.from_dict({(i,j,k,l): session_to_trodes_data[i][j][k][l] 
                           for i in session_to_trodes_data.keys() 
                           for j in session_to_trodes_data[i].keys()
                           for k in session_to_trodes_data[i][j].keys()
                           for l in session_to_trodes_data[i][j][k].keys()},
                           orient='index')

# Resetting the index and renaming the columns
trodes_metadata_df = trodes_metadata_df.reset_index()
trodes_metadata_df = trodes_metadata_df.rename(columns={'level_0': 'session_dir', 'level_1': 'recording', 'level_2': 'metadata_dir', 'level_3': 'metadata_file'}, errors="ignore")

# Adding the session path to the dataframe
trodes_metadata_df["session_path"] = trodes_metadata_df["session_dir"].map(session_to_path)

In [21]:
trodes_metadata_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_timestamp,direction,id,display_order,fields,data,filename,decimation,clock rate,session_path
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Dout1,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,3478533,output,ECU_Dout1,2,<time uint32><state uint8>,"[[3478533, 0]]",20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Dout2,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,3478533,output,ECU_Dout2,3,<time uint32><state uint8>,"[[3478533, 0]]",20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...
2,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,3478533,input,ECU_Din1,7,<time uint32><state uint8>,"[[3478533, 0], [4509412, 1], [4709411, 0], [69...",20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...
3,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Dout4,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,3478533,output,ECU_Dout4,5,<time uint32><state uint8>,"[[3478533, 0]]",20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...
4,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,3478533,input,ECU_Din2,6,<time uint32><state uint8>,"[[3478533, 0], [3581011, 1], [3583414, 0], [35...",20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...


In [22]:
trodes_metadata_df.tail()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_timestamp,direction,id,display_order,fields,data,filename,decimation,clock rate,session_path
100,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[2504265, 0, 0], [2504265, 0, 0], [2505651, 0...",20230620_114347_standard_comp_to_omission_D4_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...
101,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[2504265, 0, 0], [2504265, 0, 0], [2505651, 0...",20230620_114347_standard_comp_to_omission_D4_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...
102,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[835678, 0, 0], [837064, 0, 0], [838450, 0, 0...",20230618_100636_standard_comp_to_omission_D2_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...
103,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[835678, 0, 0], [837064, 0, 0], [838450, 0, 0...",20230618_100636_standard_comp_to_omission_D2_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...
104,20230619_115321_standard_comp_to_omission_D3_s...,20230619_115321_standard_comp_to_omission_D3_s...,video_timestamps,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[1642254, 0, 0], [1643640, 0, 0], [1645026, 0...",20230619_115321_standard_comp_to_omission_D3_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...


- Getting the first item from each tuple in the arrays in the `data` column
  - This first item is usually just the timestamp

In [23]:
trodes_metadata_df["data"].iloc[0]

array([(3478533, 0)], dtype=[('time', '<u4'), ('state', 'u1')])

In [24]:
# Getting the dtype name of each column in the numpy array
trodes_metadata_df["first_dtype_name"] = trodes_metadata_df["data"].apply(lambda x: x.dtype.names[0])
# Getting the first item of each column in the numpy array
trodes_metadata_df["first_item_data"] = trodes_metadata_df["data"].apply(lambda x: x[x.dtype.names[0]])


In [25]:
# Same as above but for the last column
trodes_metadata_df["last_dtype_name"] = trodes_metadata_df["data"].apply(lambda x: x.dtype.names[-1])
trodes_metadata_df["last_item_data"] = trodes_metadata_df["data"].apply(lambda x: x[x.dtype.names[-1]])

In [26]:
trodes_metadata_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,fields,data,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Dout1,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,<time uint32><state uint8>,"[[3478533, 0]]",20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,[3478533],state,[0]
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Dout2,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,<time uint32><state uint8>,"[[3478533, 0]]",20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,[3478533],state,[0]
2,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,<time uint32><state uint8>,"[[3478533, 0], [4509412, 1], [4709411, 0], [69...",20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[3478533, 4509412, 4709411, 6909411, 7109411, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ..."
3,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Dout4,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,<time uint32><state uint8>,"[[3478533, 0]]",20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,[3478533],state,[0]
4,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,<time uint32><state uint8>,"[[3478533, 0], [3581011, 1], [3583414, 0], [35...",20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[3478533, 3581011, 3583414, 3589014, 3626412, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ..."


In [27]:
trodes_metadata_df.tail()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,fields,data,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data
100,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[2504265, 0, 0], [2504265, 0, 0], [2505651, 0...",20230620_114347_standard_comp_to_omission_D4_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[2504265, 2504265, 2505651, 2507037, 2508422, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
101,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[2504265, 0, 0], [2504265, 0, 0], [2505651, 0...",20230620_114347_standard_comp_to_omission_D4_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[2504265, 2504265, 2505651, 2507037, 2508422, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
102,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[835678, 0, 0], [837064, 0, 0], [838450, 0, 0...",20230618_100636_standard_comp_to_omission_D2_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[835678, 837064, 838450, 838450, 839835, 84122...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
103,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[835678, 0, 0], [837064, 0, 0], [838450, 0, 0...",20230618_100636_standard_comp_to_omission_D2_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[835678, 837064, 838450, 838921, 839835, 84122...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
104,20230619_115321_standard_comp_to_omission_D3_s...,20230619_115321_standard_comp_to_omission_D3_s...,video_timestamps,4,NaN,NaN,NaN,NaN,NaN,NaN,...,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[1642254, 0, 0], [1643640, 0, 0], [1645026, 0...",20230619_115321_standard_comp_to_omission_D3_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[1642254, 1643640, 1645026, 1645026, 1646412, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [28]:
trodes_metadata_df["recording"].unique()

array(['20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged',
       '20230621_111240_standard_comp_to_omission_D5_subj_1-4_t3b3L_box1_merged',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged',
       '20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged',
       '20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged',
       '20230619_115321_standard_comp_to_omission_D3_subj_1-4_t3b3L_box2_merged',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2',
       '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1',
       '20230619_115321_standard_comp_to_omission_D3_s

## Getting the subject information from the metadata

In [29]:
trodes_metadata_df["all_subjects"] = trodes_metadata_df["session_dir"].apply(lambda x: x.replace("-","_").split("subj")[-1].split("t")[0].strip("_").replace("_",".").split(".and."))
trodes_metadata_df["all_subjects"] = trodes_metadata_df["all_subjects"].apply(lambda x: sorted([i.strip().strip(".") for i in x]))

In [30]:
trodes_metadata_df["all_subjects"]

0      [1.1, 1.2]
1      [1.1, 1.2]
2      [1.1, 1.2]
3      [1.1, 1.2]
4      [1.1, 1.2]
          ...    
100    [1.1, 1.2]
101    [1.1, 1.2]
102    [1.1, 1.4]
103    [1.1, 1.4]
104    [1.2, 1.4]
Name: all_subjects, Length: 105, dtype: object

In [31]:
trodes_metadata_df["current_subject"] = trodes_metadata_df["recording"].apply(lambda x: x.replace("-","_").split("subj")[-1].split("t")[0].strip("_").replace("_",".").split(".and.")[0])

In [32]:
trodes_metadata_df["current_subject"]

0      1.1
1      1.1
2      1.1
3      1.1
4      1.1
      ... 
100    1.2
101    1.2
102    1.4
103    1.4
104    1.2
Name: current_subject, Length: 105, dtype: object

## Dropping all the rows with unneeded metadata

In [33]:
trodes_metadata_df["metadata_dir"].unique()

array(['DIO', 'raw', 'time', 'analog', 'video_timestamps'], dtype=object)

In [34]:
METADATA_TO_KEEP = ['raw', 'DIO', 'video_timestamps']

In [35]:
trodes_metadata_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"].isin(METADATA_TO_KEEP)]

In [36]:
trodes_metadata_df = trodes_metadata_df[~trodes_metadata_df["metadata_file"].str.contains("out")]
trodes_metadata_df = trodes_metadata_df[~trodes_metadata_df["metadata_file"].str.contains("coordinates")]


In [37]:
trodes_metadata_df = trodes_metadata_df.reset_index(drop=True)

# Getting the first time stamp of each recording

In [38]:
trodes_raw_df = trodes_metadata_df[(trodes_metadata_df["metadata_dir"] == "raw") & (trodes_metadata_df["metadata_file"] == "timestamps")].copy()


In [39]:
trodes_raw_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
4,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,raw,timestamps,Raw timestamps,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[3478533, 3478534, 3478535, 3478536, 3478537, ...",time,"[3478533, 3478534, 3478535, 3478536, 3478537, ...","[1.1, 1.2]",1.1
9,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,raw,timestamps,Raw timestamps,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[3478533, 3478534, 3478535, 3478536, 3478537, ...",time,"[3478533, 3478534, 3478535, 3478536, 3478537, ...","[1.1, 1.2]",1.2
10,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,raw,timestamps,Raw timestamps,little endian,20230621_111240_standard_comp_to_omission_D5_s...,20000,2.4.0,May 24 2023,...,20230621_111240_standard_comp_to_omission_D5_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[3193044, 3193045, 3193046, 3193047, 3193048, ...",time,"[3193044, 3193045, 3193046, 3193047, 3193048, ...","[1.2, 1.4]",1.4
19,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,raw,timestamps,Raw timestamps,little endian,20230620_114347_standard_comp_to_omission_D4_s...,20000,2.4.0,May 24 2023,...,20230620_114347_standard_comp_to_omission_D4_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[2502881, 2502882, 2502883, 2502884, 2502885, ...",time,"[2502881, 2502882, 2502883, 2502884, 2502885, ...","[1.1, 1.2]",1.1
24,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,raw,timestamps,Raw timestamps,little endian,20230620_114347_standard_comp_to_omission_D4_s...,20000,2.4.0,May 24 2023,...,20230620_114347_standard_comp_to_omission_D4_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[2502881, 2502882, 2502883, 2502884, 2502885, ...",time,"[2502881, 2502882, 2502883, 2502884, 2502885, ...","[1.1, 1.2]",1.2
25,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,raw,timestamps,Raw timestamps,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,20230618_100636_standard_comp_to_omission_D2_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[835680, 835681, 835682, 835683, 835684, 83568...",time,"[835680, 835681, 835682, 835683, 835684, 83568...","[1.1, 1.4]",1.1
30,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,raw,timestamps,Raw timestamps,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,20230618_100636_standard_comp_to_omission_D2_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[835680, 835681, 835682, 835683, 835684, 83568...",time,"[835680, 835681, 835682, 835683, 835684, 83568...","[1.1, 1.4]",1.4
35,20230619_115321_standard_comp_to_omission_D3_s...,20230619_115321_standard_comp_to_omission_D3_s...,raw,timestamps,Raw timestamps,little endian,20230619_115321_standard_comp_to_omission_D3_s...,20000,2.4.0,May 24 2023,...,20230619_115321_standard_comp_to_omission_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[1642256, 1642257, 1642258, 1642259, 1642260, ...",time,"[1642256, 1642257, 1642258, 1642259, 1642260, ...","[1.2, 1.4]",1.4


In [40]:
trodes_raw_df["first_timestamp"] = trodes_raw_df["first_item_data"].apply(lambda x: x[0])

In [41]:
trodes_raw_df["recording"].iloc[0]

'20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged'

In [42]:
recording_to_first_timestamp = trodes_raw_df.set_index('session_dir')['first_timestamp'].to_dict()

In [43]:
recording_to_first_timestamp

{'20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2': 3478533,
 '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2': 3193044,
 '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1': 2502881,
 '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1': 835680,
 '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4': 1642256}

In [44]:
trodes_metadata_df["first_timestamp"] = trodes_metadata_df["session_dir"].map(recording_to_first_timestamp)

In [45]:
trodes_metadata_df["first_timestamp"]

0     3478533
1     3478533
2     3478533
3     3478533
4     3478533
5     3478533
6     3478533
7     3478533
8     3478533
9     3478533
10    3193044
11    3193044
12    3193044
13    3193044
14    3193044
15    2502881
16    2502881
17    2502881
18    2502881
19    2502881
20    2502881
21    2502881
22    2502881
23    2502881
24    2502881
25     835680
26     835680
27     835680
28     835680
29     835680
30     835680
31     835680
32     835680
33     835680
34     835680
35    1642256
36    1642256
37    1642256
38    1642256
39    1642256
40    3478533
41    3478533
42    3478533
43    3193044
44    3193044
45    2502881
46    2502881
47     835680
48     835680
49    1642256
Name: first_timestamp, dtype: int64

# Getting the event timestamps

In [46]:
trodes_metadata_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[3478533, 4509412, 4709411, 6909411, 7109411, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.2]",1.1
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[3478533, 3581011, 3583414, 3589014, 3626412, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.2]",1.1
2,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[3478533, 28509009, 34858606, 43086003, 430998...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.2]",1.1
3,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,[3478533],state,[0],"[1.1, 1.2]",1.1
4,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,raw,timestamps,Raw timestamps,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[3478533, 3478534, 3478535, 3478536, 3478537, ...",time,"[3478533, 3478534, 3478535, 3478536, 3478537, ...","[1.1, 1.2]",1.1


In [47]:
trodes_metadata_df.tail()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
45,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,20230620_114347_standard_comp_to_omission_D4_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[2504265, 2504265, 2505651, 2507037, 2508422, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.2]",1.2
46,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,20230620_114347_standard_comp_to_omission_D4_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[2504265, 2504265, 2505651, 2507037, 2508422, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.2]",1.2
47,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,20230618_100636_standard_comp_to_omission_D2_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[835678, 837064, 838450, 838450, 839835, 84122...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.4]",1.4
48,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,20230618_100636_standard_comp_to_omission_D2_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[835678, 837064, 838450, 838921, 839835, 84122...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.4]",1.4
49,20230619_115321_standard_comp_to_omission_D3_s...,20230619_115321_standard_comp_to_omission_D3_s...,video_timestamps,4,NaN,NaN,NaN,NaN,NaN,NaN,...,20230619_115321_standard_comp_to_omission_D3_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[1642254, 1643640, 1645026, 1645026, 1646412, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.2, 1.4]",1.2


In [48]:
# trodes_state_df = trodes_metadata_df[trodes_metadata_df["last_dtype_name"] == "state"].copy()

# Filtering for digital IO channels
trodes_state_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"].isin(["DIO"])].copy()
# Filtering for tone and port entry related channels
trodes_state_df = trodes_metadata_df[trodes_metadata_df["id"].isin(["ECU_Din1", "ECU_Din2"])].copy()


In [49]:
trodes_state_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[3478533, 4509412, 4709411, 6909411, 7109411, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.2]",1.1
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[3478533, 3581011, 3583414, 3589014, 3626412, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.2]",1.1
7,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[3478533, 3581011, 3583414, 3589014, 3626412, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.2]",1.2
8,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,20230617_115521_standard_comp_to_omission_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[3478533, 4509412, 4709411, 6909411, 7109411, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.2]",1.2
12,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230621_111240_standard_comp_to_omission_D5_s...,20000,2.4.0,May 24 2023,...,20230621_111240_standard_comp_to_omission_D5_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[3193044, 3416082, 3443283, 3444083, 3446685, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.2, 1.4]",1.4
14,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230621_111240_standard_comp_to_omission_D5_s...,20000,2.4.0,May 24 2023,...,20230621_111240_standard_comp_to_omission_D5_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[3193044, 3671086, 4480697, 5681112, 5881117, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.2, 1.4]",1.4
16,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230620_114347_standard_comp_to_omission_D4_s...,20000,2.4.0,May 24 2023,...,20230620_114347_standard_comp_to_omission_D4_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[2502881, 3560136, 3760141, 5960166, 6160168, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.2]",1.1
18,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230620_114347_standard_comp_to_omissio

In [50]:
trodes_state_df["event_indexes"] = trodes_state_df.apply(lambda x: np.column_stack([np.where(x["last_item_data"] == 1)[0], np.where(x["last_item_data"] == 1)[0]+1]), axis=1)

In [51]:
trodes_state_df["event_indexes"] = trodes_state_df.apply(lambda x: x["event_indexes"][x["event_indexes"][:, 1] <= x["first_item_data"].shape[0] - 1], axis=1)

In [52]:
trodes_state_df["event_timestamps"] = trodes_state_df.apply(lambda x: x["first_item_data"][x["event_indexes"]], axis=1)

## Updating the video timestamps

## Syncing up the video frame data

In [53]:
# Getting the rows that are the metadata for the video timestamps
trodes_video_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"] == "video_timestamps"].copy().reset_index(drop=True)



In [54]:
# Filtering for the first video only
# This only applies to this pilot data where we are only looking the at competition data
trodes_video_df = trodes_video_df[trodes_video_df["metadata_file"] == "1"].copy()

In [55]:
trodes_video_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
2,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,20230617_115521_standard_comp_to_omission_D1_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[3478531, 3479917, 3480033, 3481303, 3482689, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.2]",1.1
4,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,20230621_111240_standard_comp_to_omission_D5_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[3193042, 3194428, 3194428, 3195814, 3197200, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.2, 1.4]",1.4
6,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,20230620_114347_standard_comp_to_omission_D4_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[2504265, 2504265, 2505651, 2507037, 2508422, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.2]",1.2
7,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,20230618_100636_standard_comp_to_omission_D2_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[835678, 837064, 838450, 838450, 839835, 84122...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.4]",1.4


In [56]:
# Making the video timestamps into an evenly distributed array
trodes_video_df["video_timestamps"] = trodes_video_df["first_item_data"]

In [57]:
# Removing the columns that are no longer needed
trodes_video_df = trodes_video_df[["filename", "video_timestamps", "session_dir"]].copy()

In [58]:
# Renaming the filename so that we can merge with other dataframes with the same column name
trodes_video_df = trodes_video_df.rename(columns={"filename": "video_name"})

In [59]:
trodes_video_df

,video_name,video_timestamps,session_dir
2,20230617_115521_standard_comp_to_omission_D1_s...,"[3478531, 3479917, 3480033, 3481303, 3482689, ...",20230617_115521_standard_comp_to_omission_D1_s...
4,20230621_111240_standard_comp_to_omission_D5_s...,"[3193042, 3194428, 3194428, 3195814, 3197200, ...",20230621_111240_standard_comp_to_omission_D5_s...
6,20230620_114347_standard_comp_to_omission_D4_s...,"[2504265, 2504265, 2505651, 2507037, 2508422, ...",20230620_114347_standard_comp_to_omission_D4_s...
7,20230618_100636_standard_comp_to_omission_D2_s...,"[835678, 837064, 838450, 838450, 839835, 84122...",20230618_100636_standard_comp_to_omission_D2_s...


- Adding each video as a row to each state row

In [60]:
trodes_state_df = pd.merge(trodes_state_df, trodes_video_df, on=["session_dir"], how="inner")

In [61]:
trodes_state_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject,event_indexes,event_timestamps,video_name,video_timestamps
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,time,"[3478533, 4509412, 4709411, 6909411, 7109411, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.2]",1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[4509412, 4709411], [6909411, 7109411], [9209...",20230617_115521_standard_comp_to_omission_D1_s...,"[3478531, 3479917, 3480033, 3481303, 3482689, ..."
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,time,"[3478533, 3581011, 3583414, 3589014, 3626412, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.2]",1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[3581011, 3583414], [3589014, 3626412], [3671...",20230617_115521_standard_comp_to_omission_D1_s...,"[3478531, 3479917, 3480033, 3481303, 3482689, ..."
2,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,time,"[3478533, 3581011, 3583414, 3589014, 3626412, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.2]",1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[3581011, 3583414], [3589014, 3626412], [3671...",20230617_115521_standard_comp_to_omission_D1_s...,"[3478531, 3479917, 3480033, 3481303, 3482689, ..."
3,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,time,"[3478533, 4509412, 4709411, 6909411, 7109411, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.2]",1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[4509412, 4709411], [6909411, 7109411], [9209...",20230617_115521_standard_comp_to_omission_D1_s...,"[3478531, 3479917, 3480033, 3481303, 3482689, ..."
4,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230621_111240_standard_comp_to_omission_D5_s...,20000,2.4.0,May 24 2023,...,time,"[3193044, 3416082, 3443283, 3444083, 3446685, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.2, 1.4]",1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[3416082, 3443283], [3444083, 3446685], [3449...",20230621_111240_standard_comp_to_omission_D5_s...,"[3193042, 3194428, 3194428, 3195814, 3197200, ..."


## Finding the closest frame to each event

In [62]:
trodes_state_df["event_timestamps"].iloc[1]

array([[ 3581011,  3583414],
       [ 3589014,  3626412],
       [ 3671616,  3677814],
       ...,
       [83097596, 83099396],
       [83155196, 83162594],
       [83169796, 83171993]], dtype=uint32)

In [63]:
trodes_state_df["event_frames"] = trodes_state_df.apply(lambda x: utilities.helper.find_nearest_indices(x["event_timestamps"], x["video_timestamps"]), axis=1)

In [64]:
trodes_state_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject,event_indexes,event_timestamps,video_name,video_timestamps,event_frames
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,"[3478533, 4509412, 4709411, 6909411, 7109411, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.2]",1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[4509412, 4709411], [6909411, 7109411], [9209...",20230617_115521_standard_comp_to_omission_D1_s...,"[3478531, 3479917, 3480033, 3481303, 3482689, ...","[[1029, 1229], [3425, 3624], [5721, 5920], [75..."
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,"[3478533, 3581011, 3583414, 3589014, 3626412, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.2]",1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[3581011, 3583414], [3589014, 3626412], [3671...",20230617_115521_standard_comp_to_omission_D1_s...,"[3478531, 3479917, 3480033, 3481303, 3482689, ...","[[102, 105], [110, 148], [193, 199], [200, 202..."
2,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,"[3478533, 3581011, 3583414, 3589014, 3626412, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.2]",1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[3581011, 3583414], [3589014, 3626412], [3671...",20230617_115521_standard_comp_to_omission_D1_s...,"[3478531, 3479917, 3480033, 3481303, 3482689, ...","[[102, 105], [110, 148], [193, 199], [200, 202..."
3,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230617_115521_standard_comp_to_omission_D1_s...,20000,2.4.0,May 24 2023,...,"[3478533, 4509412, 4709411, 6909411, 7109411, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.2]",1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[4509412, 4709411], [6909411, 7109411], [9209...",20230617_115521_standard_comp_to_omission_D1_s...,"[3478531, 3479917, 3480033, 3481303, 3482689, ...","[[1029, 1229], [3425, 3624], [5721, 5920], [75..."
4,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230621_111240_standard_comp_to_omission_D5_s...,20000,2.4.0,May 24 2023,...,"[3193044, 3416082, 3443283, 3444083, 3446685, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.2, 1.4]",1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[3416082, 3443283], [3444083, 3446685], [3449...",20230621_111240_standard_comp_to_omission_D5_s...,"[3193042, 3194428, 3194428, 3195814, 3197200, ...","[[223, 250], [252, 254], [257, 257], [258, 297..."
5,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230621_111240_standard_comp_to_omission_D5_s...,20000,2.4.0,May 24 2023,...,"[3193044, 3671086, 4480697, 5681112, 5881117, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.2, 1.4]",1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[3671086, 4480697], [5681112, 5881117], [8081...",2

## Combine raw and state dataframes

In [65]:
trodes_state_df = trodes_state_df[STATE_COLS_TO_KEEP].drop_duplicates(subset=["session_dir", "metadata_file"]).sort_values(["session_dir", "metadata_file"]).reset_index(drop=True).copy()

In [66]:
trodes_state_df = trodes_state_df.groupby(same_columns).agg({**{col: 'first' for col in trodes_state_df.columns if col not in same_columns + different_columns}, **{col: lambda x: x.tolist() for col in different_columns}}).reset_index()

In [67]:
trodes_state_df["tone_timestamps"] = trodes_state_df["event_timestamps"].apply(lambda x: x[0])
trodes_state_df["port_entry_timestamps"] = trodes_state_df["event_timestamps"].apply(lambda x: x[1])

trodes_state_df["tone_frames"] = trodes_state_df["event_frames"].apply(lambda x: x[0])
trodes_state_df["port_entry_frames"] = trodes_state_df["event_frames"].apply(lambda x: x[1])


In [68]:
trodes_state_df = trodes_state_df.drop(columns=["event_timestamps", "event_frames"], errors="ignore")

In [69]:
trodes_state_df

,session_dir,video_name,video_timestamps,metadata_file,tone_timestamps,port_entry_timestamps,tone_frames,port_entry_frames
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[3478531, 3479917, 3480033, 3481303, 3482689, ...","[dio_ECU_Din1, dio_ECU_Din2]","[[4509412, 4709411], [6909411, 7109411], [9209...","[[3581011, 3583414], [3589014, 3626412], [3671...","[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202..."
1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[835678, 837064, 838450, 838450, 839835, 84122...","[dio_ECU_Din1, dio_ECU_Din2]","[[1934930, 2134930], [4334936, 4534932], [6634...","[[1624532, 1672532], [1674332, 1675730], [1676...","[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900..."
2,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,"[2504265, 2504265, 2505651, 2507037, 2508422, ...","[dio_ECU_Din1, dio_ECU_Din2]","[[3560136, 3760141], [5960166, 6160168], [8260...","[[2509123, 2533328], [2535126, 2543123], [2671...","[[1058, 1265], [3460, 3659], [5756, 5955], [75...","[[6, 29], [32, 40], [168, 183], [197, 218], [3..."
3,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,"[3193042, 3194428, 3194428, 3195814, 3197200, ...","[dio_ECU_Din1, dio_ECU_Din2]","[[3671086, 4480697], [5681112, 5881117], [8081...","[[3416082, 3443283], [3444083, 3446685], [3449...","[[477, 1286], [2484, 2683], [4880, 5079], [717...","[[223, 250], [252, 254], [257, 257], [258, 297..."


In [70]:
trodes_raw_df = trodes_raw_df[RAW_COLS_TO_KEEP].reset_index(drop=True).copy()

In [71]:
trodes_raw_df

,session_dir,recording,original_file,session_path,current_subject,first_item_data,first_timestamp,all_subjects
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,1.1,"[3478533, 3478534, 3478535, 3478536, 3478537, ...",3478533,"[1.1, 1.2]"
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,1.2,"[3478533, 3478534, 3478535, 3478536, 3478537, ...",3478533,"[1.1, 1.2]"
2,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,/scratch/back_up/reward_competition_extention/...,1.4,"[3193044, 3193045, 3193046, 3193047, 3193048, ...",3193044,"[1.2, 1.4]"
3,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,/scratch/back_up/reward_competition_extention/...,1.1,"[2502881, 2502882, 2502883, 2502884, 2502885, ...",2502881,"[1.1, 1.2]"
4,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,/scratch/back_up/reward_competition_extention/...,1.2,"[2502881, 2502882, 2502883, 2502884, 2502885, ...",2502881,"[1.1, 1.2]"
5,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,1.1,"[835680, 835681, 835682, 835683, 835684, 83568...",835680,"[1.1, 1.4]"
6,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,1.4,"[835680, 835681, 835682, 835683, 835684, 83568...",835680,"[1.1, 1.4]"
7,20230619_115321_standard_comp_to_omission_D3_s...,20230619_115321_standard_comp_to_omission_D3_s...,20230619_115321_standard_comp_to_omission_D3_s...,/scratch/back_up/reward_competition_extention/...,1.4,"[1642256, 1642257, 1642258, 1642259, 1642260, ...",1642256,"[1.2, 1.4]"


In [72]:
trodes_final_df = pd.merge(trodes_raw_df, trodes_state_df, on=["session_dir"], how="inner")

In [73]:
trodes_final_df = trodes_final_df.rename(columns={"first_item_data": "raw_timestamps"})
trodes_final_df = trodes_final_df.drop(columns=["metadata_file"], errors="ignore")
trodes_final_df = trodes_final_df.sort_values(["session_dir", "recording"]).reset_index(drop=True).copy()

In [74]:
sorted_columns = sorted(trodes_final_df.columns
, key=lambda x: x.split("_")[-1])
trodes_final_df = trodes_final_df[sorted_columns].copy()

## Making the timestamps 0 indexed

In [75]:
trodes_final_df[[col for col in trodes_final_df.columns if "timestamps" in col]]

,raw_timestamps,video_timestamps,tone_timestamps,port_entry_timestamps
0,"[3478533, 3478534, 3478535, 3478536, 3478537, ...","[3478531, 3479917, 3480033, 3481303, 3482689, ...","[[4509412, 4709411], [6909411, 7109411], [9209...","[[3581011, 3583414], [3589014, 3626412], [3671..."
1,"[3478533, 3478534, 3478535, 3478536, 3478537, ...","[3478531, 3479917, 3480033, 3481303, 3482689, ...","[[4509412, 4709411], [6909411, 7109411], [9209...","[[3581011, 3583414], [3589014, 3626412], [3671..."
2,"[835680, 835681, 835682, 835683, 835684, 83568...","[835678, 837064, 838450, 838450, 839835, 84122...","[[1934930, 2134930], [4334936, 4534932], [6634...","[[1624532, 1672532], [1674332, 1675730], [1676..."
3,"[835680, 835681, 835682, 835683, 835684, 83568...","[835678, 837064, 838450, 838450, 839835, 84122...","[[1934930, 2134930], [4334936, 4534932], [6634...","[[1624532, 1672532], [1674332, 1675730], [1676..."
4,"[2502881, 2502882, 2502883, 2502884, 2502885, ...","[2504265, 2504265, 2505651, 2507037, 2508422, ...","[[3560136, 3760141], [5960166, 6160168], [8260...","[[2509123, 2533328], [2535126, 2543123], [2671..."
5,"[2502881, 2502882, 2502883, 2502884, 2502885, ...","[2504265, 2504265, 2505651, 2507037, 2508422, ...","[[3560136, 3760141], [5960166, 6160168], [8260...","[[2509123, 2533328], [2535126, 2543123], [2671..."
6,"[3193044, 3193045, 3193046, 3193047, 3193048, ...","[3193042, 3194428, 3194428, 3195814, 3197200, ...","[[3671086, 4480697], [5681112, 5881117], [8081...","[[3416082, 3443283], [3444083, 3446685], [3449..."


In [76]:
trodes_final_df

,session_dir,original_file,tone_frames,port_entry_frames,video_name,session_path,recording,current_subject,all_subjects,first_timestamp,raw_timestamps,video_timestamps,tone_timestamps,port_entry_timestamps
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,"[1.1, 1.2]",3478533,"[3478533, 3478534, 3478535, 3478536, 3478537, ...","[3478531, 3479917, 3480033, 3481303, 3482689, ...","[[4509412, 4709411], [6909411, 7109411], [9209...","[[3581011, 3583414], [3589014, 3626412], [3671..."
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,"[1.1, 1.2]",3478533,"[3478533, 3478534, 3478535, 3478536, 3478537, ...","[3478531, 3479917, 3480033, 3481303, 3482689, ...","[[4509412, 4709411], [6909411, 7109411], [9209...","[[3581011, 3583414], [3589014, 3626412], [3671..."
2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,"[1.1, 1.4]",835680,"[835680, 835681, 835682, 835683, 835684, 83568...","[835678, 837064, 838450, 838450, 839835, 84122...","[[1934930, 2134930], [4334936, 4534932], [6634...","[[1624532, 1672532], [1674332, 1675730], [1676..."
3,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,"[1.1, 1.4]",835680,"[835680, 835681, 835682, 835683, 835684, 83568...","[835678, 837064, 838450, 838450, 839835, 84122...","[[1934930, 2134930], [4334936, 4534932], [6634...","[[1624532, 1672532], [1674332, 1675730], [1676..."
4,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,"[[1058, 1265], [3460, 3659], [5756, 5955], [75...","[[6, 29], [32, 40], [168, 183], [197, 218], [3...",20230620_114347_standard_comp_to_omission_D4_s...,/scratch/back_up/reward_competition_extention/...,20230620_114347_standard_comp_to_omission_D4_s...,1.1,"[1.1, 1.2]",2502881,"[2502881, 2502882, 2502883, 2502884, 2502885, ...","[2504265, 2504265, 2505651, 2507037, 2508422, ...","[[3560136, 3760141], [5960166, 6160168], [8260...","[[2509123, 2533328], [2535126, 2543123], [2671..."
5,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,"[[1058, 1265], [3460, 3659], [5756, 5955], [75...","[[6, 29], [32, 40], [168, 183], [197, 218], [3...",20230620_114347_standard_comp_to_omission_D4_s...,/scratch/back_up/reward_competition_extention/...,20230620_114347_standard_comp_to_omission_D4_s...,1.2,"[1.1, 1.2]",2502881,"[2502881, 2502882, 2502883, 2502884, 2502885, ...","[2504265, 2504265, 2505651, 2507037, 2508422, ...","[[3560136, 3760141], [5960166, 6160168], [8260...","[[2509123, 2533328], [2535126, 2543123], [2671..."
6,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,"[[477, 1286], [2484, 2683], [4880, 5079], [717...","[[223, 250], [252, 254], [257, 257], [258, 297...",20230621_111240_standard_comp_to_omission_D5_s...,/scrat

In [77]:
for col in [col for col in trodes_final_df.columns if "timestamps" in col]:
    trodes_final_df[col] = trodes_final_df.apply(lambda x: x[col].astype(np.int32) - np.int32(x["first_timestamp"]), axis=1)

for col in [col for col in trodes_final_df.columns if "frames" in col]:
    trodes_final_df[col] = trodes_final_df[col].apply(lambda x: x.astype(np.int32))

In [78]:
trodes_final_df.head()

,session_dir,original_file,tone_frames,port_entry_frames,video_name,session_path,recording,current_subject,all_subjects,first_timestamp,raw_timestamps,video_timestamps,tone_timestamps,port_entry_timestamps
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,"[1.1, 1.2]",3478533,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 1500, 2770, 4156, 5542, 5542, 6927,...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ..."
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,"[1.1, 1.2]",3478533,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 1500, 2770, 4156, 5542, 5542, 6927,...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ..."
2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,"[1.1, 1.4]",835680,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 2770, 4155, 5541, 6927, 6927,...","[[1099250, 1299250], [3499256, 3699252], [5799...","[[788852, 836852], [838652, 840050], [840652, ..."
3,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,"[1.1, 1.4]",835680,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 2770, 4155, 5541, 6927, 6927,...","[[1099250, 1299250], [3499256, 3699252], [5799...","[[788852, 836852], [838652, 840050], [840652, ..."
4,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,"[[1058, 1265], [3460, 3659], [5756, 5955], [75...","[[6, 29], [32, 40], [168, 183], [197, 218], [3...",20230620_114347_standard_comp_to_omission_D4_s...,/scratch/back_up/reward_competition_extention/...,20230620_114347_standard_comp_to_omission_D4_s...,1.1,"[1.1, 1.2]",2502881,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 1384, 2770, 4156, 5541, 5541, 6927, 831...","[[1057255, 1257260], [3457285, 3657287], [5757...","[[6242, 30447], [32245, 40242], [169046, 18424..."


## Saving to a file

In [79]:
trodes_final_df.to_pickle(os.path.join(OUTPUT_DIR, "{}_trodes_metadata.pkl".format(OUTPUT_PREFIX)))